# Analyse der PV-Einspeisevergütungen in der Schweiz
Dieses Notebook untersucht die Entwicklung und Verteilung der Einspeisevergütungen für Photovoltaik-Anlagen in der Schweiz.

## Datenimport, Aufbereitung und Anreicherung
In diesem Abschnitt werden die relevanten Datensätze geladen, bereinigt und für die Analyse vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd

# EVU- und Gemeindedaten laden
evu_df = pd.read_csv('../data/vese/evu_ergebnisse.csv', sep=';')  # Daten der Energieversorger (EVU)
gemeinde_df = pd.read_csv('../data/vese/gemeinde_ergebnisse.csv', sep=';')  # Gemeindedaten

# Daten anhand der ElCom-Nummer zusammenführen (inner join)
merged_df = pd.merge(evu_df, gemeinde_df, on='nrElcom', how='inner')

# Gemeindestand für Kantons- und BFS-Zuordnung laden
gemeindestand_df = pd.read_csv('../data/vese/Gemeindestand.csv', sep=';')

# Mapping: BFS-Gemeindenummer → Kanton
gemeinde_to_kanton = gemeindestand_df.set_index('BFS Gde-nummer')['Kanton'].to_dict()

# Kantonsspalte anhand der Gemeindenummer zuordnen
merged_df['Kanton'] = merged_df['idofs'].map(gemeinde_to_kanton)

# Fehlende Werte in den Vergütungskomponenten mit 0 ersetzen
merged_df = merged_df.copy()  # Kopie für sicheres Arbeiten
merged_df['energy1'] = merged_df['energy1'].fillna(0)
merged_df['eco1'] = merged_df['eco1'].fillna(0)

# Zeilen entfernen, bei denen keine Vergütung gezahlt wird
merged_df = merged_df[~((merged_df['energy1'] == 0) & (merged_df['eco1'] == 0))]

# Gesamteinspeisevergütung berechnen (Rp/kWh)
merged_df['remuneration'] = merged_df['energy1'] + merged_df['eco1']

## Interaktives Balkendiagramm: Durchschnittliche Vergütung pro Kanton und Jahr

In [ ]:
import plotly.express as px

# Durchschnittliche Vergütung pro Jahr und Kanton berechnen
df = (
    merged_df
    .groupby(['year', 'Kanton'])['remuneration']
    .mean()
    .reset_index()
)
df.rename(columns={'remuneration': 'avg_remuneration'}, inplace=True)
df['avg_remuneration'] = df['avg_remuneration'].round(2)

# Für jedes Jahr die Top-4 Kantone mit höchster Vergütung bestimmen
top_cantons_per_year = (
    df.groupby('year', group_keys=False)
    .apply(lambda g: g.nlargest(4, 'avg_remuneration')['Kanton'].tolist())
    .to_dict()
)

def assign_color_group(row, top_dict):
    """Hilfsfunktion zur Gruppenzuordnung für die Farbgebung."""
    return row['Kanton'] if row['Kanton'] in top_dict.get(row['year'], []) else 'Other'

df['canton_group'] = df.apply(assign_color_group, axis=1, top_dict=top_cantons_per_year)

# Interaktives Balkendiagramm mit Jahr-Schieberegler erstellen
fig = px.bar(
    df,
    x='Kanton',
    y='avg_remuneration',
    animation_frame='year',
    color='canton_group',
    text='avg_remuneration',
    labels={'avg_remuneration': 'Durchschnittliche Vergütung (Rp/kWh)', 'Kanton': 'Kanton'},
    color_discrete_map={'Other': '#D9D9D9'},
    template='simple_white'
)

fig.update_traces(texttemplate='%{text:.0f}', textposition='inside')

fig.update_layout(
    xaxis=dict(categoryorder='total descending', title='Kanton'),
    yaxis=dict(range=[0, 35], title='Durchschnittliche Vergütung (Rp/kWh)'),
    title='Durchschnittliche Einspeisevergütung pro Kanton und Jahr',
    showlegend=False
)

fig.show()
fig.write_html("../docs/assets/diagramme/pv_verguetungen_sortiert_korrekt.html")

## Choroplethenkarte: Vergütung der PV-Anlagen pro Gemeinde (2024)

In [ ]:
import folium
import json
from shapely.geometry import shape, mapping
from shapely.ops import transform
from functools import partial
import pyproj

# Daten für das Jahr 2024 filtern
df = merged_df[merged_df['year'] == 2024].copy()

# Durchschnittliche Vergütung pro Gemeinde berechnen
cumulative_map_data = df.groupby(['idofs'])['remuneration'].mean().reset_index()

# GeoJSON mit Gemeindeflächen laden und Geometrie vereinfachen
with open('../data/electricity_production_plants/gemeinden.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Fehlende Gemeinden mit Durchschnittswert auffüllen
average_remuneration = cumulative_map_data['remuneration'].mean()
missing_bfs = set(
    feature['properties']['gemeinde_BFS_NUMMER'] for feature in geojson_data['features']
) - set(cumulative_map_data['idofs'])
for bfs in missing_bfs:
    cumulative_map_data = pd.concat([
        cumulative_map_data,
        pd.DataFrame({'idofs': [bfs], 'remuneration': [average_remuneration]})
    ], ignore_index=True)

# Geometrien vereinfachen und Vergütungswerte zuordnen
simplified_features = []
for feature in geojson_data['features']:
    geom = shape(feature['geometry'])
    simplify = partial(
        pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:4326', always_xy=True).transform
    )
    simplified_geom = transform(simplify, geom).simplify(0.001, preserve_topology=True)
    feature['geometry'] = mapping(simplified_geom)
    bfs_nr = feature['properties']['gemeinde_BFS_NUMMER']
    remuneration = (
        cumulative_map_data.loc[cumulative_map_data['idofs'] == bfs_nr, 'remuneration'].values[0]
        if bfs_nr in cumulative_map_data['idofs'].values else 0
    )
    feature['properties']['remuneration'] = f"{remuneration:.2f}"
    feature['properties']['bfs-str'] = str(bfs_nr)
    simplified_features.append(feature)

geojson_data['features'] = simplified_features

# Karte initialisieren
map = folium.Map(location=[46.8, 8.33], zoom_start=7)

# Choroplethen-Layer hinzufügen
folium.Choropleth(
    geo_data=geojson_data,
    name='Choropleth',
    data=cumulative_map_data,
    columns=['idofs', 'remuneration'],
    key_on='feature.properties.gemeinde_BFS_NUMMER',
    fill_color='RdYlGn',
    fill_opacity=0.9,
    line_opacity=0.6,
    line_color='black',
    legend_name='Vergütung (Rp/kWh)',
    threshold_scale=[0, 5, 10, 15, 20, 25, cumulative_map_data['remuneration'].max()],
).add_to(map)

# Tooltips und Gemeindegrenzen hinzufügen
folium.GeoJson(
    geojson_data,
    name="Gemeinden",
    style_function=lambda feature: {
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['gemeinde_NAME', 'bfs-str', 'remuneration'],
        aliases=['Gemeinde:', 'BFS-Nummer:', 'Vergütung (Rp/kWh):'],
        localize=True
    )
).add_to(map)

# Titel einfügen und Karte speichern
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 15%;"><b>Choroplethenkarte: PV-Einspeisevergütung pro Gemeinde (2024)</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

map.save('../docs/assets/diagramme/map_pv_vergütungen.html')
map